# Computer Vision Fundamentals - Interactive Learning Notebook 👁️🤖

Welcome to your hands-on computer vision journey! This notebook provides interactive examples and exercises to master computer vision concepts.

## 🎯 What You'll Learn

1. **Image Processing Basics**: Loading, displaying, and manipulating images
2. **Feature Detection**: Finding important patterns in images  
3. **Object Detection**: Identifying and locating objects
4. **Real-world Applications**: Building practical computer vision systems

## 🛠️ Setup

First, let's import all necessary libraries and set up our environment.

In [ ]:
# Install required packages (run this cell first)
!pip install opencv-python matplotlib numpy pillow scikit-image
!pip install ultralytics  # For YOLO object detection

print("✅ Installation complete!")

In [ ]:
# Import libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import requests
from io import BytesIO
import warnings
warnings.filterwarnings('ignore')

# Set up matplotlib for better display
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['image.cmap'] = 'gray'

print("📚 Libraries imported successfully!")
print(f"OpenCV version: {cv2.__version__}")

## 📸 Section 1: Image Processing Fundamentals

Let's start with the basics - how computers "see" and process images.

In [ ]:
# Function to load a sample image from the internet
def load_sample_image(url=None):
    """Load a sample image for experimentation"""
    
    if url is None:
        # Use a sample image URL (feel free to replace with your own)
        url = "https://upload.wikimedia.org/wikipedia/commons/thumb/5/50/Vd-Orig.png/256px-Vd-Orig.png"
    
    try:
        response = requests.get(url)
        image = Image.open(BytesIO(response.content))
        # Convert to OpenCV format (BGR)
        image_cv = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        return image_cv
    except:
        # Create a simple test image if download fails
        print("Creating a test image...")
        test_image = np.zeros((300, 400, 3), dtype=np.uint8)
        cv2.rectangle(test_image, (50, 50), (150, 150), (0, 255, 0), -1)
        cv2.circle(test_image, (300, 100), 50, (255, 0, 0), -1)
        cv2.putText(test_image, 'Test Image', (50, 250), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        return test_image

# Load our sample image
sample_image = load_sample_image()
print(f"✅ Image loaded! Shape: {sample_image.shape}")
print(f"   - Height: {sample_image.shape[0]} pixels")
print(f"   - Width: {sample_image.shape[1]} pixels")
print(f"   - Channels: {sample_image.shape[2]} (BGR format)")

In [ ]:
# Display the image
def display_image(image, title="Image", cmap=None):
    """Helper function to display images properly"""
    plt.figure(figsize=(10, 6))
    
    if len(image.shape) == 3:
        # Color image - convert BGR to RGB for display
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        plt.imshow(image_rgb)
    else:
        # Grayscale image
        plt.imshow(image, cmap='gray')
    
    plt.title(title)
    plt.axis('off')
    plt.show()

# Display our sample image
display_image(sample_image, "Our Sample Image")

### 🎨 Understanding Image Properties

Let's explore what makes up a digital image.

In [ ]:
# Analyze image properties
def analyze_image_properties(image):
    """Analyze and display various image properties"""
    
    print("🔍 Image Analysis:")
    print("=" * 40)
    
    # Basic properties
    print(f"📐 Dimensions: {image.shape}")
    print(f"💾 Data type: {image.dtype}")
    print(f"📊 Total pixels: {image.size:,}")
    print(f"💭 Memory usage: {image.nbytes / (1024*1024):.2f} MB")
    
    # Pixel value statistics
    print(f"\n📈 Pixel Value Statistics:")
    print(f"   Minimum value: {image.min()}")
    print(f"   Maximum value: {image.max()}")
    print(f"   Mean value: {image.mean():.2f}")
    print(f"   Standard deviation: {image.std():.2f}")
    
    # Create visualization
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    # Original image
    axes[0, 0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    axes[0, 0].set_title('Original Image')
    axes[0, 0].axis('off')
    
    # Color channels
    b, g, r = cv2.split(image)
    
    axes[0, 1].imshow(r, cmap='Reds')
    axes[0, 1].set_title('Red Channel')
    axes[0, 1].axis('off')
    
    axes[0, 2].imshow(g, cmap='Greens')
    axes[0, 2].set_title('Green Channel')
    axes[0, 2].axis('off')
    
    axes[1, 0].imshow(b, cmap='Blues')
    axes[1, 0].set_title('Blue Channel')
    axes[1, 0].axis('off')
    
    # Histogram
    colors = ['red', 'green', 'blue']
    for i, color in enumerate(colors):
        hist = cv2.calcHist([image], [i], None, [256], [0, 256])
        axes[1, 1].plot(hist, color=color, alpha=0.7)
    axes[1, 1].set_title('Color Histogram')
    axes[1, 1].set_xlabel('Pixel Value')
    axes[1, 1].set_ylabel('Frequency')
    
    # Grayscale version
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    axes[1, 2].imshow(gray, cmap='gray')
    axes[1, 2].set_title('Grayscale')
    axes[1, 2].axis('off')
    
    plt.tight_layout()
    plt.show()

# Analyze our sample image
analyze_image_properties(sample_image)

### 🔧 Basic Image Operations

Now let's learn to transform and enhance images.

In [ ]:
# Basic image transformations
def demonstrate_transformations(image):
    """Demonstrate basic image transformations"""
    
    # Resize
    resized = cv2.resize(image, (300, 200))
    
    # Rotate
    height, width = image.shape[:2]
    rotation_matrix = cv2.getRotationMatrix2D((width/2, height/2), 45, 1)
    rotated = cv2.warpAffine(image, rotation_matrix, (width, height))
    
    # Flip
    flipped_horizontal = cv2.flip(image, 1)
    flipped_vertical = cv2.flip(image, 0)
    
    # Brightness adjustment
    brighter = cv2.convertScaleAbs(image, alpha=1.0, beta=50)
    darker = cv2.convertScaleAbs(image, alpha=1.0, beta=-50)
    
    # Display results
    transformations = [
        (image, 'Original'),
        (resized, 'Resized'),
        (rotated, 'Rotated 45°'),
        (flipped_horizontal, 'Flipped Horizontal'),
        (brighter, 'Brighter'),
        (darker, 'Darker')
    ]
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()
    
    for i, (img, title) in enumerate(transformations):
        if len(img.shape) == 3:
            img_display = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        else:
            img_display = img
        
        axes[i].imshow(img_display)
        axes[i].set_title(title)
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

# Demonstrate transformations
demonstrate_transformations(sample_image)

### 🎛️ Image Filtering and Enhancement

Learn to improve image quality and extract important features.

In [ ]:
# Image filtering demonstrations
def demonstrate_filtering(image):
    """Demonstrate various image filters"""
    
    # Convert to grayscale for some operations
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Gaussian blur
    blurred = cv2.GaussianBlur(image, (15, 15), 0)
    
    # Sharpening
    sharpening_kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    sharpened = cv2.filter2D(image, -1, sharpening_kernel)
    
    # Edge detection
    edges_canny = cv2.Canny(gray, 50, 150)
    edges_sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 1, ksize=3)
    edges_sobel = np.uint8(np.absolute(edges_sobel))
    
    # Noise reduction
    denoised = cv2.medianBlur(image, 5)
    
    # Display results
    filters = [
        (image, 'Original', 'color'),
        (blurred, 'Gaussian Blur', 'color'),
        (sharpened, 'Sharpened', 'color'),
        (edges_canny, 'Canny Edges', 'gray'),
        (edges_sobel, 'Sobel Edges', 'gray'),
        (denoised, 'Denoised', 'color')
    ]
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()
    
    for i, (img, title, img_type) in enumerate(filters):
        if img_type == 'color':
            img_display = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            axes[i].imshow(img_display)
        else:
            axes[i].imshow(img, cmap='gray')
        
        axes[i].set_title(title)
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

# Demonstrate filtering
demonstrate_filtering(sample_image)

## 🔍 Section 2: Feature Detection

Now let's find important patterns and landmarks in images.

In [ ]:
# Corner detection demonstration
def detect_corners(image):
    """Demonstrate corner detection methods"""
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Harris corner detection
    harris_corners = cv2.cornerHarris(gray, 2, 3, 0.04)
    harris_corners = cv2.dilate(harris_corners, None)
    
    # Shi-Tomasi corner detection  
    shi_tomasi_corners = cv2.goodFeaturesToTrack(gray, maxCorners=100, qualityLevel=0.01, minDistance=10)
    
    # FAST corner detection
    fast = cv2.FastFeatureDetector_create()
    fast_keypoints = fast.detect(gray, None)
    
    # Visualize results
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # Harris corners
    harris_img = image.copy()
    harris_img[harris_corners > 0.01 * harris_corners.max()] = [0, 0, 255]
    axes[0].imshow(cv2.cvtColor(harris_img, cv2.COLOR_BGR2RGB))
    axes[0].set_title('Harris Corner Detection')
    axes[0].axis('off')
    
    # Shi-Tomasi corners
    shi_tomasi_img = image.copy()
    if shi_tomasi_corners is not None:
        for corner in shi_tomasi_corners:
            x, y = corner.ravel()
            cv2.circle(shi_tomasi_img, (int(x), int(y)), 3, (0, 255, 0), -1)
    axes[1].imshow(cv2.cvtColor(shi_tomasi_img, cv2.COLOR_BGR2RGB))
    axes[1].set_title(f'Shi-Tomasi ({len(shi_tomasi_corners) if shi_tomasi_corners is not None else 0} corners)')
    axes[1].axis('off')
    
    # FAST corners
    fast_img = cv2.drawKeypoints(image, fast_keypoints, None, color=(255, 0, 0))
    axes[2].imshow(cv2.cvtColor(fast_img, cv2.COLOR_BGR2RGB))
    axes[2].set_title(f'FAST ({len(fast_keypoints)} keypoints)')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("🎯 Corner Detection Results:")
    print(f"   Harris corners: Found distinct corner regions")
    print(f"   Shi-Tomasi: {len(shi_tomasi_corners) if shi_tomasi_corners is not None else 0} good corners")
    print(f"   FAST: {len(fast_keypoints)} fast keypoints")

# Detect corners in our sample image
detect_corners(sample_image)

## 🎯 Section 3: Object Detection

Let's build systems that can identify and locate objects in images.

In [ ]:
# Object detection with YOLO
try:
    from ultralytics import YOLO
    
    def detect_objects_yolo(image):
        """Detect objects using YOLO"""
        
        print("🔄 Loading YOLO model...")
        model = YOLO('yolov8n.pt')  # This will download the model if not present
        
        print("🔍 Running object detection...")
        results = model(image, verbose=False)
        
        # Process results
        detections = []
        annotated_image = image.copy()
        
        for r in results:
            if r.boxes is not None:
                for box in r.boxes:
                    # Extract detection info
                    x1, y1, x2, y2 = box.xyxy[0].tolist()
                    confidence = float(box.conf[0])
                    class_id = int(box.cls[0])
                    class_name = model.names[class_id]
                    
                    detections.append({
                        'class': class_name,
                        'confidence': confidence,
                        'bbox': [x1, y1, x2, y2]
                    })
                    
                    # Draw bounding box
                    cv2.rectangle(annotated_image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                    
                    # Draw label
                    label = f"{class_name}: {confidence:.2f}"
                    label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]
                    cv2.rectangle(annotated_image, (int(x1), int(y1) - label_size[1] - 10), 
                                 (int(x1) + label_size[0], int(y1)), (0, 255, 0), -1)
                    cv2.putText(annotated_image, label, (int(x1), int(y1) - 5), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
        
        # Display results
        fig, axes = plt.subplots(1, 2, figsize=(20, 8))
        
        axes[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        axes[0].set_title('Original Image')
        axes[0].axis('off')
        
        axes[1].imshow(cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB))
        axes[1].set_title(f'Object Detection Results ({len(detections)} objects)')
        axes[1].axis('off')
        
        plt.tight_layout()
        plt.show()
        
        # Print detection summary
        if detections:
            print("🎯 Detection Results:")
            for i, det in enumerate(detections, 1):
                print(f"   {i}. {det['class']}: {det['confidence']:.2f} confidence")
        else:
            print("ℹ️  No objects detected in this image.")
        
        return detections, annotated_image
    
    # Run object detection
    detections, annotated = detect_objects_yolo(sample_image)
    
except ImportError:
    print("⚠️  YOLO not available. Install with: pip install ultralytics")
    print("For now, let's demonstrate a simple object detection concept...")
    
    def simple_circle_detection(image):
        """Detect circles as a simple object detection example"""
        
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Use HoughCircles to detect circular objects
        circles = cv2.HoughCircles(
            gray, cv2.HOUGH_GRADIENT, dp=1, minDist=50,
            param1=50, param2=30, minRadius=10, maxRadius=100
        )
        
        result_image = image.copy()
        
        if circles is not None:
            circles = np.round(circles[0, :]).astype("int")
            
            for (x, y, r) in circles:
                cv2.circle(result_image, (x, y), r, (0, 255, 0), 4)
                cv2.circle(result_image, (x, y), 2, (0, 0, 255), 3)
        
        # Display results
        fig, axes = plt.subplots(1, 2, figsize=(15, 6))
        
        axes[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        axes[0].set_title('Original Image')
        axes[0].axis('off')
        
        axes[1].imshow(cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB))
        axes[1].set_title(f'Circle Detection ({len(circles) if circles is not None else 0} circles)')
        axes[1].axis('off')
        
        plt.tight_layout()
        plt.show()
        
        print(f"🔍 Found {len(circles) if circles is not None else 0} circular objects")
    
    simple_circle_detection(sample_image)

## 🛠️ Section 4: Building Your Own Tools

Let's create practical computer vision applications.

In [ ]:
# Interactive Image Editor
class InteractiveImageEditor:
    """A simple interactive image editor"""
    
    def __init__(self, image):
        self.original = image.copy()
        self.current = image.copy()
        self.history = []
    
    def reset(self):
        """Reset to original image"""
        self.current = self.original.copy()
        self.history = []
        print("✅ Reset to original image")
    
    def apply_blur(self, kernel_size=15):
        """Apply Gaussian blur"""
        self.current = cv2.GaussianBlur(self.current, (kernel_size, kernel_size), 0)
        self.history.append(f"Blur (kernel={kernel_size})")
        print(f"✅ Applied blur with kernel size {kernel_size}")
    
    def adjust_brightness(self, value=0):
        """Adjust brightness"""
        self.current = cv2.convertScaleAbs(self.current, alpha=1.0, beta=value)
        self.history.append(f"Brightness {value:+d}")
        print(f"✅ Adjusted brightness by {value:+d}")
    
    def adjust_contrast(self, value=1.0):
        """Adjust contrast"""
        self.current = cv2.convertScaleAbs(self.current, alpha=value, beta=0)
        self.history.append(f"Contrast x{value}")
        print(f"✅ Adjusted contrast by factor of {value}")
    
    def convert_to_grayscale(self):
        """Convert to grayscale"""
        gray = cv2.cvtColor(self.current, cv2.COLOR_BGR2GRAY)
        self.current = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
        self.history.append("Grayscale conversion")
        print("✅ Converted to grayscale")
    
    def apply_edge_detection(self):
        """Apply edge detection"""
        gray = cv2.cvtColor(self.current, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, 50, 150)
        self.current = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
        self.history.append("Edge detection")
        print("✅ Applied edge detection")
    
    def show_current(self):
        """Display current image"""
        display_image(self.current, "Current Image")
    
    def show_comparison(self):
        """Show before/after comparison"""
        fig, axes = plt.subplots(1, 2, figsize=(15, 6))
        
        axes[0].imshow(cv2.cvtColor(self.original, cv2.COLOR_BGR2RGB))
        axes[0].set_title('Original')
        axes[0].axis('off')
        
        axes[1].imshow(cv2.cvtColor(self.current, cv2.COLOR_BGR2RGB))
        axes[1].set_title('Edited')
        axes[1].axis('off')
        
        plt.tight_layout()
        plt.show()
        
        if self.history:
            print("📝 Edit History:")
            for i, edit in enumerate(self.history, 1):
                print(f"   {i}. {edit}")

# Create an image editor instance
editor = InteractiveImageEditor(sample_image)

print("🎨 Interactive Image Editor Created!")
print("Try these commands:")
print("   editor.apply_blur(15)")
print("   editor.adjust_brightness(50)")
print("   editor.adjust_contrast(1.5)")
print("   editor.convert_to_grayscale()")
print("   editor.apply_edge_detection()")
print("   editor.show_current()")
print("   editor.show_comparison()")
print("   editor.reset()")

In [ ]:
# Try out the image editor!
# Experiment with different combinations

# Example 1: Create a dramatic effect
editor.reset()
editor.adjust_contrast(1.3)
editor.adjust_brightness(20)
editor.show_comparison()

In [ ]:
# Example 2: Create an artistic edge effect
editor.reset()
editor.apply_blur(5)
editor.apply_edge_detection()
editor.show_comparison()

## 🎯 Practice Exercises

Now it's your turn! Try these exercises to reinforce your learning.

In [ ]:
# Exercise 1: Color Space Explorer
def exercise_color_spaces():
    """
    Exercise: Explore different color spaces
    
    TODO: Complete this function to:
    1. Convert the sample image to HSV color space
    2. Split into H, S, V channels
    3. Display each channel separately
    4. Try to isolate a specific color range
    """
    
    print("🎨 Exercise 1: Color Space Explorer")
    print("Your task: Convert image to HSV and explore channels")
    
    # TODO: Your code here
    # Hints:
    # - Use cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # - Use cv2.split() to separate channels
    # - Use cv2.inRange() to isolate color ranges
    
    pass

# Uncomment to try the exercise
# exercise_color_spaces()

In [ ]:
# Exercise 2: Build a Motion Detector
def exercise_motion_detection():
    """
    Exercise: Create a simple motion detection system
    
    TODO: Complete this function to:
    1. Take two images (or frames)
    2. Calculate the difference
    3. Threshold the difference to find motion
    4. Highlight motion areas
    """
    
    print("🏃 Exercise 2: Motion Detection")
    print("Your task: Detect differences between two images")
    
    # Create two slightly different images for demo
    img1 = sample_image.copy()
    img2 = sample_image.copy()
    
    # Add some "motion" to the second image
    cv2.circle(img2, (100, 100), 30, (255, 255, 255), -1)
    
    # TODO: Your code here
    # Hints:
    # - Convert images to grayscale
    # - Use cv2.absdiff() to find differences
    # - Use cv2.threshold() to create binary mask
    # - Use cv2.findContours() to find motion regions
    
    pass

# Uncomment to try the exercise
# exercise_motion_detection()

## 🚀 Advanced Challenge: Build a Feature Matcher

Ready for a challenge? Let's build a system that can match features between two images.

In [ ]:
# Advanced Challenge: Feature Matching
def feature_matching_challenge():
    """
    Advanced Challenge: Build a feature matching system
    
    This demonstrates how to find corresponding points between two images
    """
    
    print("🎯 Advanced Challenge: Feature Matching")
    
    # Create two versions of the same image (simulating different viewpoints)
    img1 = sample_image.copy()
    
    # Transform the second image slightly
    rows, cols = img1.shape[:2]
    M = cv2.getRotationMatrix2D((cols/2, rows/2), 15, 0.9)
    img2 = cv2.warpAffine(img1, M, (cols, rows))
    
    # Convert to grayscale
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    
    try:
        # Initialize SIFT detector
        sift = cv2.SIFT_create()
        
        # Find keypoints and descriptors
        kp1, des1 = sift.detectAndCompute(gray1, None)
        kp2, des2 = sift.detectAndCompute(gray2, None)
        
        print(f"🔍 Found {len(kp1)} features in image 1")
        print(f"🔍 Found {len(kp2)} features in image 2")
        
        # Match features
        bf = cv2.BFMatcher()
        matches = bf.knnMatch(des1, des2, k=2)
        
        # Apply ratio test to filter good matches
        good_matches = []
        for m, n in matches:
            if m.distance < 0.75 * n.distance:
                good_matches.append([m])
        
        print(f"✅ Found {len(good_matches)} good matches")
        
        # Draw matches
        img_matches = cv2.drawMatchesKnn(
            img1, kp1, img2, kp2, good_matches, None,
            flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
        )
        
        # Display result
        plt.figure(figsize=(20, 8))
        plt.imshow(cv2.cvtColor(img_matches, cv2.COLOR_BGR2RGB))
        plt.title(f'Feature Matching: {len(good_matches)} good matches')
        plt.axis('off')
        plt.show()
        
        # Show individual images with keypoints
        fig, axes = plt.subplots(1, 2, figsize=(15, 6))
        
        img1_kp = cv2.drawKeypoints(img1, kp1, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        img2_kp = cv2.drawKeypoints(img2, kp2, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        
        axes[0].imshow(cv2.cvtColor(img1_kp, cv2.COLOR_BGR2RGB))
        axes[0].set_title(f'Image 1: {len(kp1)} keypoints')
        axes[0].axis('off')
        
        axes[1].imshow(cv2.cvtColor(img2_kp, cv2.COLOR_BGR2RGB))
        axes[1].set_title(f'Image 2: {len(kp2)} keypoints')
        axes[1].axis('off')
        
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"⚠️  Error in feature matching: {e}")
        print("This might be due to OpenCV version or image characteristics")

# Run the feature matching challenge
feature_matching_challenge()

## 🎓 Congratulations!

You've completed the Computer Vision Fundamentals notebook! 🎉

### What You've Learned:

1. **Image Basics**: How computers represent and process images
2. **Transformations**: Resizing, rotating, and enhancing images
3. **Filtering**: Applying various filters for enhancement and feature extraction
4. **Feature Detection**: Finding important patterns and landmarks
5. **Object Detection**: Identifying and locating objects in images
6. **Practical Applications**: Building real computer vision tools

### Next Steps:

1. **Experiment More**: Try the exercises with your own images
2. **Build Projects**: Create your own computer vision applications
3. **Learn Advanced Topics**: Explore deep learning for computer vision
4. **Join Communities**: Connect with other computer vision enthusiasts

### Resources for Continued Learning:

- **OpenCV Documentation**: https://docs.opencv.org/
- **Computer Vision Courses**: Coursera, edX, Udacity
- **Research Papers**: arXiv.org computer vision section
- **Competitions**: Kaggle computer vision competitions

Keep experimenting and building! Computer vision is a rapidly evolving field with endless possibilities. 🚀

In [ ]:
# Final challenge: Create your own computer vision function!
def your_custom_cv_function(image):
    """
    Your turn! Create a custom computer vision function.
    
    Ideas:
    - Image quality scorer
    - Artistic filter creator
    - Simple object counter
    - Color palette extractor
    
    Be creative!
    """
    
    # TODO: Your creative computer vision code here!
    
    print("🎨 Your custom computer vision function!")
    print("Add your own creative image processing here.")
    
    # Example: Simple color palette extractor
    # Reshape image to a list of pixels
    pixels = image.reshape(-1, 3)
    
    # Convert to float for better processing
    pixels = np.float32(pixels)
    
    # Use k-means to find dominant colors
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 1.0)
    k = 5
    _, labels, centers = cv2.kmeans(pixels, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    
    # Convert back to uint8
    centers = np.uint8(centers)
    
    print(f"🎨 Dominant colors in the image:")
    for i, color in enumerate(centers):
        print(f"   Color {i+1}: RGB({color[2]}, {color[1]}, {color[0]})")
    
    return centers

# Try your custom function
palette = your_custom_cv_function(sample_image)